In [2]:
from fastapi import FastAPI, Body
import requests
import pandas as pd
import boto3
import os
from io import BytesIO
import boto3
from dotenv import load_dotenv

In [33]:
load_dotenv(r'e:\Data\ML-engineer\mle-project-sprint-4-v001\flask\.env')

True

In [40]:
# Создаем клиент s3
s3 = boto3.client('s3', endpoint_url='https://storage.yandexcloud.net', aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'), aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'))

# Проверяем возможность подключения к s3 бакету
try:
    s3.head_bucket(Bucket=os.getenv("S3_BUCKET"))
    print("Подключение к s3 бакету успешно")
except Exception as e:
    print("Ошибка подключения к s3 бакету:", e)

Подключение к s3 бакету успешно


In [43]:
buffer = BytesIO()
# Get the file inside the S3 Bucket
s3_response = s3.download_fileobj(
    Bucket=os.getenv("S3_BUCKET"),
    Key='recsys/data/items.parquet',
    Fileobj=buffer
)
buffer.seek(0)

# Read the data in bytes format
df = pd.read_parquet(buffer)

In [54]:
def get_tracks(user_id: int):
    # Отправляем запрос к другому микросервису
    response = requests.get(f"http://127.0.0.1:7000/get_tracks?user_id={user_id}")
    track_list = response.json()
    return track_list
    # Находим строки, где track_id is in track_list
    #result = df[df['track_id'].isin(track_list)]
    #return result.to_dict(orient='records')

In [72]:
def get_history(user_id: int):
    # Отправляем запрос к другому микросервису
    response = requests.get(f"http://127.0.0.1:8000/get_tracks?user_id={user_id}")
    track_list = response.json()
    return track_list

In [111]:
def similars(track_ids):
    # Отправляем запрос к другому микросервису
    response = requests.get("http://127.0.0.1:6000/get_tracks", json=track_ids)
    track_list = response.json()
    return track_list

In [116]:
def get_pops(track_ids):
    # Отправляем запрос к другому микросервису
    response = requests.get("http://127.0.0.1:8000/get_pop_tracks", json=track_ids)
    track_list = response.json()
    return track_list

In [117]:
rex = get_tracks(13)
hix = get_history(13)
sims = similars(hix['data'])
pops = get_pops(hix['data'])
rex, hix, sims, pops

({'status': 'success',
  'data': [63678480,
   63678470,
   63678479,
   64453581,
   9282507,
   24112220,
   39632975,
   17887582,
   560468,
   2200948]},
 {'status': 'success',
  'data': [64453581,
   63678480,
   63678479,
   63678470,
   63549849,
   62487201,
   63276502,
   60729121,
   60406441,
   60089337]},
 {'status': 'success',
  'data': [63678475,
   63750588,
   63678469,
   63678476,
   63678477,
   63750579,
   63678471,
   63710488,
   63678478,
   63750585]},
 {'status': 'success',
  'data': [308971,
   648937,
   8264,
   10449127,
   14662829,
   202851,
   33146010,
   2180613,
   9563114,
   35515789]})

In [118]:
if len(rex['data']) >=7:
    out = rex['data'][:7]+sims['data'][:3]
if len(rex['data']) ==0:
    if len(hix['data'])==10:
        out =sims['data'][:10]
    else:
        out = pops


In [125]:
cool_result = [(row['track_name']+"-"+row['artists']) for _, row in df.query('track_id in @out').iterrows()]
cool_result

["('Родина',)-['АнимациЯ']",
 "('Трасса Е-95',)-['Алиса']",
 "('Две тысячи баксов',)-['Алексей Марковников']",
 "('Весна',)-['Игорь Растеряев']",
 "('Песня про Юру Прищепного',)-['Игорь Растеряев']",
 "('Казачья',)-['Игорь Растеряев']",
 "('Комбайнёры',)-['Игорь Растеряев']",
 "('Георгиевская ленточка',)-['Игорь Растеряев']",
 "('Дальнобойная',)-['Игорь Растеряев']",
 "('Байконур 66',)-['Без Билета']"]

In [3]:
events = pd.read_parquet(r'E:\Data\ML-engineer\mle-project-sprint-4-v001\data\events.parquet')
cb_pred = pd.read_parquet(r'E:\Data\ML-engineer\mle-project-sprint-4-v001\data\cb_predictions.parquet')

In [4]:
users_with_history = list(events['user_id'].unique())
users_with_predictions = list(cb_pred['user_id'].unique())

In [7]:
predicted_users = set(users_with_predictions)
users_with_history_and_no_predictions = []
for user in users_with_history:
    if user not in predicted_users:
        users_with_history_and_no_predictions.append(user)
        if len(users_with_history_and_no_predictions) >= 10:
            break


In [8]:
users_with_history_and_no_predictions

[]